In [99]:
#USPS preprocessing

import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import PCA
# time to import the data!
df_usps_train = pd.read_csv('zip.train',delim_whitespace=True,header=None)
df_usps_test = pd.read_csv('zip.test',delim_whitespace=True,header=None)

In [100]:
#see the data a bit

print(df_usps_train.head())
print(df_usps_train.shape)
print(df_usps_test.shape)

   0    1    2    3      4      5      6      7      8      9    ...    247  \
0  6.0 -1.0 -1.0 -1.0 -1.000 -1.000 -1.000 -1.000 -0.631  0.862  ...  0.304   
1  5.0 -1.0 -1.0 -1.0 -0.813 -0.671 -0.809 -0.887 -0.671 -0.853  ... -0.671   
2  4.0 -1.0 -1.0 -1.0 -1.000 -1.000 -1.000 -1.000 -1.000 -1.000  ... -1.000   
3  7.0 -1.0 -1.0 -1.0 -1.000 -1.000 -0.273  0.684  0.960  0.450  ... -0.318   
4  3.0 -1.0 -1.0 -1.0 -1.000 -1.000 -0.928 -0.204  0.751  0.466  ...  0.466   

     248    249    250    251    252    253    254    255  256  
0  0.823  1.000  0.482 -0.474 -0.991 -1.000 -1.000 -1.000 -1.0  
1 -0.671 -0.033  0.761  0.762  0.126 -0.095 -0.671 -0.828 -1.0  
2 -1.000 -1.000 -0.109  1.000 -0.179 -1.000 -1.000 -1.000 -1.0  
3  1.000  0.536 -0.987 -1.000 -1.000 -1.000 -1.000 -1.000 -1.0  
4  0.639  1.000  1.000  0.791  0.439 -0.199 -0.883 -1.000 -1.0  

[5 rows x 257 columns]
(7291, 257)
(2007, 257)


In [101]:
#train on USPS
#label data as -1 iff digit <=4, else 1
df_usps_modified = df_usps_train.copy()
df_usps_sample_0 = df_usps_modified.loc[df_usps_modified.loc[:,0]==0].sample(200,random_state=seed_value)
df_usps_sample_1 = df_usps_modified.loc[df_usps_modified.loc[:,0]==1].sample(200,random_state=seed_value)
df_usps_sample_2 = df_usps_modified.loc[df_usps_modified.loc[:,0]==2].sample(200,random_state=seed_value)
df_usps_sample_3 = df_usps_modified.loc[df_usps_modified.loc[:,0]==3].sample(200,random_state=seed_value)
df_usps_sample_4 = df_usps_modified.loc[df_usps_modified.loc[:,0]==4].sample(200,random_state=seed_value)
df_usps_sample_5 = df_usps_modified.loc[df_usps_modified.loc[:,0]==5].sample(200,random_state=seed_value)
df_usps_sample_6 = df_usps_modified.loc[df_usps_modified.loc[:,0]==6].sample(200,random_state=seed_value)
df_usps_sample_7 = df_usps_modified.loc[df_usps_modified.loc[:,0]==7].sample(200,random_state=seed_value)
df_usps_sample_8 = df_usps_modified.loc[df_usps_modified.loc[:,0]==8].sample(200,random_state=seed_value)
df_usps_sample_9 = df_usps_modified.loc[df_usps_modified.loc[:,0]==9].sample(200,random_state=seed_value)

df_usps_sample = pd.concat((df_usps_sample_0,df_usps_sample_1,df_usps_sample_2,df_usps_sample_3,df_usps_sample_4,df_usps_sample_5,df_usps_sample_6,df_usps_sample_7,df_usps_sample_8,df_usps_sample_9))

for i in range(df_usps_modified.shape[0]):
    if df_usps_modified.loc[i,0] <=4:
        df_usps_modified.loc[i,0] = -1
    else:
        df_usps_modified.loc[i,0] = 1


df_usps_modified_test = df_usps_test.copy()
for i in range(df_usps_modified_test.shape[0]):
    if df_usps_modified_test.loc[i,0] <=4:
        df_usps_modified_test.loc[i,0] = -1
    else:
        df_usps_modified_test.loc[i,0] = 1
#print(df_usps_modified)
#print(df_usps_modified_test)
print(len(df_usps_modified[df_usps_modified.loc[:,0]==1].index))

3051


In [102]:
'''for i in range(1000):
    seed_value=i
    #sample 2000 from them
    df_usps_sample = df_usps_modified.sample(2000,random_state=seed_value)

    #print(df_usps_sample)
    size = len(df_usps_sample[df_usps_sample.loc[:,0]==1].index)
    #print(size)
    if (np.abs(size-1000)<115):
        print(seed_value)'''
        
#print(len(df_usps_sample[df_usps_sample.loc[:,0]==8].index))
#print(df_usps_sample)

'for i in range(1000):\n    seed_value=i\n    #sample 2000 from them\n    df_usps_sample = df_usps_modified.sample(2000,random_state=seed_value)\n\n    #print(df_usps_sample)\n    size = len(df_usps_sample[df_usps_sample.loc[:,0]==1].index)\n    #print(size)\n    if (np.abs(size-1000)<115):\n        print(seed_value)'

In [103]:
#method of partitioning data fram in 50*40
seed_value=83
def partition_dataframe(df):
    result=[]
    df_to_sample_from = df.copy()
    for i in range(0,50):
        train0 = df_to_sample_from[df_to_sample_from.loc[:,0]==0].sample(4,random_state=seed_value)
        train1 = df_to_sample_from[df_to_sample_from.loc[:,0]==1].sample(4,random_state=seed_value)
        train2 = df_to_sample_from[df_to_sample_from.loc[:,0]==2].sample(4,random_state=seed_value)
        train3 = df_to_sample_from[df_to_sample_from.loc[:,0]==3].sample(4,random_state=seed_value)
        train4 = df_to_sample_from[df_to_sample_from.loc[:,0]==4].sample(4,random_state=seed_value)
        train5 = df_to_sample_from[df_to_sample_from.loc[:,0]==5].sample(4,random_state=seed_value)
        train6 = df_to_sample_from[df_to_sample_from.loc[:,0]==6].sample(4,random_state=seed_value)
        train7 = df_to_sample_from[df_to_sample_from.loc[:,0]==7].sample(4,random_state=seed_value)
        train8 = df_to_sample_from[df_to_sample_from.loc[:,0]==8].sample(4,random_state=seed_value)
        train9 = df_to_sample_from[df_to_sample_from.loc[:,0]==9].sample(4,random_state=seed_value)
        train = pd.concat((train0,train1,train2,train3,train4,train5,train6,train7,train8,train9))
        remaining = df_to_sample_from.drop(train.index)
        df_to_sample_from = remaining
        result.append(train)
    return result
#store the sample
partition = partition_dataframe(df_usps_sample)


In [104]:
print(df_usps_sample)
for i in range(df_usps_sample.shape[0]):
    if df_usps_sample.iloc[i,0] <=4:
        df_usps_sample.iloc[i,0] = -1
    else:
        df_usps_sample.iloc[i,0] = 1
print(df_usps_sample)

      0    1    2      3      4      5      6      7      8      9    ...  \
7068  0.0 -1.0 -1.0 -1.000 -1.000 -1.000 -1.000 -1.000 -0.969  0.305  ...   
370   0.0 -1.0 -1.0 -1.000 -1.000 -1.000 -1.000 -0.878 -0.074  0.977  ...   
4751  0.0 -1.0 -1.0 -1.000 -0.951 -0.452  0.201  0.201  0.199 -0.469  ...   
1852  0.0 -1.0 -1.0 -0.663 -0.263 -0.477 -1.000 -1.000 -1.000 -1.000  ...   
3582  0.0 -1.0 -1.0 -1.000 -1.000 -1.000 -0.904 -0.004  0.291  0.967  ...   
...   ...  ...  ...    ...    ...    ...    ...    ...    ...    ...  ...   
1791  9.0 -1.0 -1.0 -1.000 -1.000 -1.000 -1.000 -0.477  0.813  1.000  ...   
6728  9.0 -1.0 -1.0 -1.000 -1.000 -1.000 -1.000 -1.000 -0.939 -0.320  ...   
2969  9.0 -1.0 -1.0 -1.000 -1.000 -1.000 -0.864 -0.043  0.544  0.759  ...   
3448  9.0 -1.0 -1.0 -1.000 -1.000 -1.000 -1.000 -1.000 -0.870  0.520  ...   
3381  9.0 -1.0 -1.0 -0.947 -0.362 -0.020  0.418  0.905  1.000  1.000  ...   

        247    248    249    250    251    252  253  254  255  256  
7068 -

In [105]:
'''dataframe_file = [partition,df_usps_sample, df_usps_modified_test]
with open('usbs_databases', 'wb') as fp:
    pickle.dump(dataframe_file, fp)'''

for i in range(0,len(partition)):
    for j in range(partition[i].shape[0]):
        if partition[i].iloc[j,0] <=4:
            partition[i].iloc[j,0] = -1
        else:
            partition[i].iloc[j,0] = 1


In [106]:
dataframe_file = [partition,df_usps_sample, df_usps_modified_test]
with open('usbs_databases', 'wb') as fp:
    pickle.dump(dataframe_file, fp)